In [9]:
import pandas as pd
import numpy as np
import pdb
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
# from scipy.misc import imread
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import lightgbm as gbm
from sklearn.decomposition import PCA
import os
import pickle

%matplotlib nbagg

In [5]:
train_data = pd.read_csv('TrainDataScaledPCA.csv', index_col=0)

In [7]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,default_ind
application_key,,,,,,,,,,,,,,,,,,,,,
230032,7.708774,12.346500,-2.144122,2.495861,-4.746134,-2.385384,-4.641845,0.773270,-1.294483,1.278017,...,0.115893,0.123205,0.417220,-0.041836,0.116702,0.344066,0.022961,-0.085918,0.113807,0.0
230033,-5.544693,-2.049064,-4.138349,0.614378,1.721533,-0.639281,-1.719649,-0.838887,-1.209555,0.702528,...,-0.056883,-0.038358,-0.019811,0.235915,-0.155602,0.302034,-0.098669,0.399895,-0.285475,1.0
230034,-5.525315,-2.315925,-1.997191,1.495989,-2.272323,1.711687,-1.662008,1.097928,0.145318,-0.700140,...,-0.123000,0.367400,-0.416936,0.262710,0.261290,-0.285663,0.320851,-0.555613,0.249159,1.0
230035,-5.972098,-2.600077,-1.988013,1.968241,-0.431336,0.343533,-1.084700,1.180114,-1.331916,-0.355266,...,0.324805,-0.170804,0.114337,-0.006339,-0.153569,0.135306,0.118619,-0.022996,0.195353,0.0
230036,-6.765522,-3.210112,-2.506669,3.129291,0.565960,0.817904,-1.269161,1.217510,-0.440333,1.105082,...,-0.122154,-0.248032,-0.479094,-0.254767,0.043486,0.317568,-0.359390,-0.085078,-0.060012,0.0


In [18]:
models_XGB_LB = []

# Parameters to be input directly
t = [0.5, 0.5, None, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.45, 0.45, None, 0.45, 0.45, None, 0.5, None, 0.5]
n = [600, 500, 400, 50, 300, 100, 100, 300, 100, 50, 500, 200, 100, 50, 400, 500, 500, 200, 400, 50]
max_depth = [2, 2, 3, 10, 3, 10, 5, 3, 15, 10, 2, 1, 1, 1, 2, 2, 2, 3, 2, 15]
models = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]

model_name = ['LightBoost', 'AdaBoost']

n_undersamples = len(models) 
i = 0


for i in range(n_undersamples):
    
    pos_class = train_data[train_data['default_ind']==1]
    neg_class = train_data[train_data['default_ind']==0]
    neg_resampled = neg_class.sample(n= int(1 * len(pos_class)), replace=False)

    train_data_resampled = pd.concat([pos_class, neg_resampled])

    # Use these for Trainings

    X_train_resampled = train_data_resampled.drop('default_ind', axis=1)
    y_train_resampled = train_data_resampled['default_ind']    

    # Under Sampling Negative Class
    print(round(i/n_undersamples*100, 3), '% Done')
    print('\n\nt =', t[i], 'Number of Estimators =', n[i], 'Depth =', max_depth[i], 'Model =', model_name[models[i]])

    #Training

    if models[i] == 1:
    
        #XGBoost
        model_XGB = XGBClassifier(n_estimators = n[i] , learning_rate=0.1, max_depth=max_depth[i] ,reg_lambda=0.01, n_jobs = -1, verbose =100) 
        model_XGB.fit(X_train_resampled, y_train_resampled)

        y_hat_XGB = model_XGB.predict(X_train_resampled)
        print('\nTrain Score XGB =', accuracy_score(y_train_resampled, y_hat_XGB))
        print('Train F1 Score XGB =', f1_score(y_train_resampled, y_hat_XGB))    
        print('Train Precision XGB=', precision_score(y_train_resampled, y_hat_XGB))
        print('Train Recall XGB=', recall_score(y_train_resampled, y_hat_XGB))

        models_XGB_LB.append(model_XGB)
    
    else:
    
        #LightBoost
        cv_params = {
        'max_depth': max_depth[i],
        'objective': 'binary',
        'metric':'auc',  
        'feature_fraction': 1, 
        'bagging_fraction': 0.75,
        'reg_lambda': 1,
        'n_estimators': n[i]
        }

        gbm_train = gbm.Dataset(X_train_resampled, y_train_resampled)
        model_LB = gbm.train(cv_params,  
                    gbm_train,
                    verbose_eval=1)

        y_train_prob = model_LB.predict(X_train_resampled)
        thres = t[i]
        y_pred_train = np.zeros(len(y_train_prob))
        y_pred_train[np.argwhere(y_train_prob>thres)] = 1
        print('\nTrain Score LB =', accuracy_score(y_train_resampled, y_pred_train))
        print('Train F1 Score LB =', f1_score(y_train_resampled, y_pred_train))
        print('Train Precision LB=', precision_score(y_train_resampled, y_pred_train))
        print('Train Recall LB=', recall_score(y_train_resampled, y_pred_train))

        models_XGB_LB.append(model_LB)

0.0 % Done


t = 0.5 Number of Estimators = 600 Depth = 2 Model = LightBoost

Train Score LB = 0.7340101522842639
Train F1 Score LB = 0.7349407747755341
Train Precision LB= 0.7323783920020163
Train Recall LB= 0.7375211505922166
5.0 % Done


t = 0.5 Number of Estimators = 500 Depth = 2 Model = LightBoost

Train Score LB = 0.728891708967851
Train F1 Score LB = 0.7313914501257335
Train Precision LB= 0.7247093023255814
Train Recall LB= 0.7381979695431472
10.0 % Done


t = None Number of Estimators = 400 Depth = 3 Model = AdaBoost

Train Score XGB = 0.7544627749576989
Train F1 Score XGB = 0.7543432718962271
Train Precision XGB= 0.7547105898293602
Train Recall XGB= 0.7539763113367174
15.0 % Done


t = 0.5 Number of Estimators = 50 Depth = 10 Model = LightBoost

Train Score LB = 0.7419204737732656
Train F1 Score LB = 0.7436339188167073
Train Precision LB= 0.7387293371180498
Train Recall LB= 0.7486040609137056
20.0 % Done


t = 0.5 Number of Estimators = 300 Depth = 3 Model = LightBoost

Train

In [21]:
models_all = np.array(models_XGB_LB)
pickle.dump(models_all, open('models.sav', 'wb'))
pickle.dump(t, open('thresholds.sav', 'wb'))